# Understand Agentic architecture concept with AutoGen Studio

----
AutoGen Studio offers four main interfaces to help you build and manage multi-agent systems:

1. Team Builder
* A visual interface for creating agent teams through declarative specification (JSON) or drag-and-drop
* Supports configuration of all core components: teams, agents, tools, models, and termination conditions
* Fully compatible with AgentChat’s component definitions
2. Playground
* Interactive environment for testing and running agent teams
* Features include:
    * Live message streaming between agents
    * Visual representation of message flow through a control transition graph
    * Interactive sessions with teams using UserProxyAgent
    * Full run control with the ability to pause or stop execution
3. Gallery
* Central hub for discovering and importing community-created components
* Enables easy integration of third-party components
4. Deployment
* Export and run teams in python code
* Setup and test endpoints based on a team configuration
* Run teams in a docker container

**Reference**
- [AutoGen Studio repo](https://github.com/microsoft/autogen/tree/main/python/packages/autogen-studio) 

In [1]:
from dotenv import load_dotenv
import os
import json
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.ai.evaluation import GroundednessEvaluator, RelevanceEvaluator, RetrievalEvaluator
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core.models import ChatCompletionClient, SystemMessage, UserMessage, AssistantMessage
from autogen_core import MessageContext, RoutedAgent, SingleThreadedAgentRuntime, TopicId, message_handler, type_subscription
from pydantic import BaseModel
from typing import List
from dataclasses import dataclass


load_dotenv("../../../.env")

False

In [1]:
!pip install -U autogenstudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 17.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 51.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.6 MB/s eta 0:00:00:00

In [ ]:
# you can do it in the terminal as well
!autogenstudio ui --port 8081

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:297: PydanticDeprecatedSince20: `json_encoders` is deprecated. See https://docs.pydantic.dev/2.10/concepts/serialization/#custom-serializers for alternatives. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(
2025-02-05 09:49:24.359 | INFO     | autogenstudio.web.initialization:__init__:40 - Initialized application data folder: /home/azureuser/.autogenstudio
INFO:     Started server process [22733]
INFO:     Waiting for appli

## 🧪 Step 1. Set up components in your gallery 
---

1. Go to the started localhost:{your port} website.
2. Click on the `Gallery` menu.
3. Click on the `+ New Gallery` button to add a new component.
4. Upload the customized sample_gallery.json file. 
5. Check the 11 components in the gallery.

## 🧪 Step 2. Single Agent with Tools
---

1. Go to the Team Builder menu and Click on the `+ New Team` button.
2. Remove the default agent and termination condition.
3. Drag and drop the `single_tool_use_agent` and `CombinationTemination` in the team box.
4. Click the save button and goes to the Playground menu.
5. Click the `+ New Session` button to start a session and input the name of session and select your team created.
6. Click the `Create` button to run the session.
7. input your query in the input box and click the `Send` button.

![Single-Agent Workflow](images/single-agent.png)

### 🚀 Example of your messages

1. What is the history of microsoft?
2. get info from https://www.sqlite.org/copyright.html
3. what is the weather in Seoul?
4. 4.11 + 5.11?

### ✅ sample response

1. What is the history of microsoft?

![screenshot1](images/screenshot1.png)

2. get info from https://www.sqlite.org/copyright.html

![screenshot2](images/screenshot2.png)

3. what is the weather in Seoul?

![screenshot3](images/screenshot3.png)

4. 4.11 + 5.11?

![screenshot4](images/screenshot4.png)



## 🧪 Step 3. Multi-Agent (Writer and Reviewer)
---

1. Go to the Team Builder menu and Click on the `+ New Team` button.
2. Remove the default agent and termination condition.
3. Drag and drop the `writer_agent`, `reviwer_agent` and `CombinationTemination` in the team box.
4. Click the save button and goes to the Playground menu.
5. Click the `+ New Session` button to start a session and input the name of session and select your team created.
6. Click the `Create` button to run the session.
7. input your query in the input box and click the `Send` button.

![Multi-Agent Workflow](images/multi-agent.png)

### 🚀 Example of your messages

1. what is the height of the eiffel tower
2. make a bedtime story for 5 years old kids
2. make a bedtime story for 10 years old kids

### ✅ sample response

1. make a bedtime story for 10 years old kids

![screenshot5](images/screenshot5.png)

### 